# Exploration des données — Electio-Analytics

Notebook d'exploration de la base SQLite `electio_herault.db`.  
Données filtrées pour le département de l'Hérault (34), 341 communes.

**Prérequis** : avoir exécuté le pipeline ETL (`python main.py etl`) pour générer la base.

In [ ]:
import sqlite3
import pandas as pd

DB_PATH = "../data/output/electio_herault.db"
conn = sqlite3.connect(DB_PATH)
print(f"Connecté à {DB_PATH}")

## 1. Vue d'ensemble de la base

12 tables, toutes liées par `codgeo` (code INSEE commune).

In [ ]:
# Lister toutes les tables et leur nombre de lignes
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)

print(f"{'Table':<25s} {'Lignes':>8s}")
print("-" * 35)
for table in tables['name']:
    count = pd.read_sql(f"SELECT COUNT(*) as nb FROM {table}", conn)['nb'][0]
    print(f"  {table:<23s} {count:>8,}")

## 2. Communes (table de référence)

Table pivot du modèle : chaque commune a un `codgeo` unique (code INSEE 5 caractères).

In [ ]:
communes = pd.read_sql("SELECT * FROM communes", conn)
print(f"Nombre de communes : {len(communes)}")
print(f"Colonnes : {list(communes.columns)}")
communes.sample(10)

## 3. Élections municipales

Résultats par candidat, par commune, pour les municipales 2008, 2014, 2020.  
Chaque candidat est classé `Gauche` ou `Droite` (colonne `camp`).

In [ ]:
elections = pd.read_sql("SELECT * FROM elections", conn)
print(f"Lignes : {len(elections):,}")
print(f"Années : {sorted(elections['annee'].unique())}")
print(f"Tours : {sorted(elections['tour'].unique())}")
print(f"\nRépartition par camp :")
print(elections['camp'].value_counts())
elections.head()

In [ ]:
# Répartition des voix par année et camp (T1 uniquement)
t1 = elections[elections['tour'] == 1]
pivot = t1.groupby(['annee', 'camp'])['voix'].sum().unstack(fill_value=0)
pivot['total'] = pivot.sum(axis=1)
pivot['pct_gauche'] = (100 * pivot.get('Gauche', 0) / pivot['total']).round(1)
pivot

## 4. Population

Population municipale par commune et par année de recensement (1968 à 2022).

In [ ]:
population = pd.read_sql("SELECT * FROM population", conn)
print(f"Lignes : {len(population):,}")
print(f"Années disponibles : {sorted(population['annee'].unique())}")
print(f"\nStatistiques sur la population :")
population['population'].describe().round(0)

In [ ]:
# Top 10 communes les plus peuplées (dernière année disponible)
derniere_annee = population['annee'].max()
top_pop = population[population['annee'] == derniere_annee].nlargest(10, 'population')
top_pop = top_pop.merge(communes[['codgeo', 'nom']], on='codgeo', how='left')
print(f"Top 10 communes en {derniere_annee} :")
top_pop[['nom', 'codgeo', 'population']]

## 5. Revenus

In [ ]:
revenus = pd.read_sql("SELECT * FROM revenus", conn)
print(f"Lignes : {len(revenus)}")
print(f"Colonnes ({len(revenus.columns)}) :")
for c in revenus.columns:
    print(f"  - {c}")
revenus.head()

## 6. CSP (Catégories socio-professionnelles)

Actifs de 25-54 ans par catégorie (cadres, ouvriers, employés, etc.) et par année de recensement.

In [ ]:
csp = pd.read_sql("SELECT * FROM csp", conn)
print(f"Lignes : {len(csp)}")
print(f"Années RP : {sorted(csp['annee'].unique())}")
print(f"\nColonnes ({len(csp.columns)}) :")
for c in csp.columns[:15]:
    print(f"  - {c}")
if len(csp.columns) > 15:
    print(f"  ... et {len(csp.columns) - 15} autres")

## 7. Diplômes

Niveau de diplôme de la population de 15 ans et plus, par commune.  
Préfixes : `p11` (RP 2011), `p16` (RP 2016), `p22` (RP 2022).

In [ ]:
diplomes = pd.read_sql("SELECT * FROM diplomes", conn)
print(f"Lignes : {len(diplomes)}")
print(f"\nColonnes ({len(diplomes.columns)}) :")
for c in diplomes.columns[:20]:
    print(f"  - {c}")
if len(diplomes.columns) > 20:
    print(f"  ... et {len(diplomes.columns) - 20} autres")

## 8. Comptes des communes

Données financières : dette, recettes, dépenses de fonctionnement et d'investissement.

In [ ]:
comptes = pd.read_sql("SELECT * FROM comptes_communes", conn)
print(f"Lignes : {len(comptes)}")
print(f"Années : {sorted(comptes['annee'].unique())}")
print(f"Colonnes : {list(comptes.columns)}")
comptes.describe().round(0)

## 9. Catastrophes naturelles (CatNat)

In [ ]:
catnat = pd.read_sql("SELECT * FROM catnat", conn)
print(f"Lignes : {len(catnat)}")
print(f"Communes touchées : {catnat['codgeo'].nunique()}")
print(f"Colonnes : {list(catnat.columns)}")

# Top 10 communes les plus touchées
top_cat = catnat.groupby('codgeo').size().sort_values(ascending=False).head(10)
noms = communes.set_index('codgeo')['nom']
top_cat.index = top_cat.index.map(lambda x: f"{noms.get(x, x)} ({x})")
print(f"\nTop 10 communes les plus touchées :")
print(top_cat.to_string())

## 10. Qualité des données

Vérification des valeurs manquantes et de la couverture par table.

In [ ]:
tables_list = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)

for table in tables_list['name']:
    df = pd.read_sql(f"SELECT * FROM {table}", conn)
    nulls = df.isnull().sum()
    total_nulls = nulls.sum()
    if total_nulls > 0:
        print(f"\n{table} ({len(df)} lignes) — {total_nulls} valeurs manquantes :")
        for col, n in nulls[nulls > 0].items():
            print(f"  {col}: {n} ({100*n/len(df):.1f}%)")
    else:
        print(f"{table} ({len(df)} lignes) — OK")

In [ ]:
# Vérifier la couverture des jointures : combien de communes ont des données dans chaque table
codgeo_communes = set(communes['codgeo'])
tables_avec_codgeo = ['elections', 'population', 'revenus', 'csp', 'diplomes',
                       'comptes_communes', 'catnat', 'naissances_deces']

print(f"Communes de référence : {len(codgeo_communes)}")
print(f"\n{'Table':<25s} {'Communes':>10s} {'Couverture':>12s}")
print("-" * 50)
for table in tables_avec_codgeo:
    try:
        codgeos = set(pd.read_sql(f"SELECT DISTINCT codgeo FROM {table}", conn)['codgeo'])
        inter = codgeos & codgeo_communes
        pct = 100 * len(inter) / len(codgeo_communes)
        print(f"  {table:<23s} {len(inter):>8d}   {pct:>8.1f}%")
    except Exception as e:
        print(f"  {table:<23s} Erreur: {e}")

In [ ]:
conn.close()
print("Connexion fermée.")